In [9]:
# Import necessary libraries
import cv2  # For image processing
import pytesseract  # For Optical Character Recognition (OCR)
import numpy as np  # For array operations


In [10]:
# Set the path to the Tesseract-OCR executable
# Ensure you have Tesseract installed at this path
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"



In [11]:
# Load Haar Cascade classifier for license plate detection
plate_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_russian_plate_number.xml")



In [12]:
def preprocess_image(image):
    """
    Preprocess the input image to improve plate detection.
    - Convert the image to grayscale.
    - Equalize the histogram to handle lighting variations.
    """
    if image is None:
        print("Error: Image could not be loaded.")
        return None
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)
    return gray


In [13]:
def recognize_plate(plate_img):
    """
    Perform OCR on the cropped license plate image.
    - Use Tesseract with custom configurations for better accuracy.
    """
    plate_text = pytesseract.image_to_string(plate_img, config='--psm 8 --oem 3')
    return plate_text.strip()  # Remove any leading or trailing spaces


In [14]:
def detect_plate_in_image(image_path):
    """
    Detect license plates in an image and annotate the detected plates.
    - Draw rectangles around detected plates.
    - Use OCR to recognize and display the license plate text.
    """
    img = cv2.imread(image_path)
    if img is None:
        print("Error: Could not read image.")
        return

    # Preprocess the image
    gray = preprocess_image(img)
    if gray is None:
        return

    # Detect license plates in the image
    plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))

    for (x, y, w, h) in plates:
        # Draw a rectangle around the detected plate
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # Extract the plate region and recognize text
        plate_img = img[y:y + h, x:x + w]
        plate_text = recognize_plate(plate_img)
        
        # Annotate the image with the recognized text
        cv2.putText(img, f'Plate: {plate_text}', (x - 3, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the annotated image
    cv2.imshow('Number Plate Recognition', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [15]:
def detect_plate_in_video(video_path):
    """
    Detect license plates in a video and annotate frames in real time.
    - Draw rectangles around detected plates in each frame.
    - Use OCR to recognize and display license plate text.
    """
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = video.read()
        if not ret:
            print("Error: Failed to read video frame.")
            break

        # Preprocess the current frame
        gray = preprocess_image(frame)
        if gray is None:
            continue

        # Detect license plates in the frame
        plates = plate_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5, minSize=(25, 25))

        for (x, y, w, h) in plates:
            # Draw a rectangle around the detected plate
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            
            # Extract the plate region and recognize text
            plate_img = frame[y:y + h, x:x + w]
            plate_text = recognize_plate(plate_img)
            
            # Annotate the frame with the recognized text
            cv2.putText(frame, f'Plate: {plate_text}', (x - 3, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the annotated frame
        cv2.imshow('Video Plate Recognition', frame)

        # Press 'q' to stop the video
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()


In [20]:
# Run the detection on an image
detect_plate_in_image('images/Cars415.png')

# Run the detection on a video
detect_plate_in_video('vid.mp4')
